In [1]:
import cftime
import numpy as np
import xarray as xr
import climpred
import proplot as plot
import glob
from tqdm import tqdm
import dask.array as da
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FixedLocator
import warnings
warnings.filterwarnings("ignore")

from dask.distributed import Client
import dask.config
dask.config.set({"array.slicing.split_large_chunks": False})

In [2]:
client = Client("tcp://10.12.205.14:36622")

In [3]:
verifdata = "CPC" #can be ERA5 or CPC
#domain = "global"
domain = "North America"
metric = "rps" #can be rps, crps, crpss, rmse, acc
if metric == "acc" or metric=="rmse":
    comp="e2o"
    dim="init"
else:
    comp="m2o"
    dim=["init","member"]

In [4]:
hinda2 = xr.open_zarr("/glade/campaign/cesm/development/cross-wg/S2S/jaye/CESM2.S2S.tas_2m.anoms.wklead.zarr/", consolidated=True)
hinda1 = xr.open_zarr("/glade/campaign/cesm/development/cross-wg/S2S/jaye/CESM1.S2S.tas_2m.anoms.wklead.zarr/", consolidated=True)
hindaw = xr.open_zarr("/glade/campaign/cesm/development/cross-wg/S2S/jaye/WACCM.S2S.tas_2m.anoms.wklead.zarr/", consolidated=True)
verif = xr.open_zarr("/glade/campaign/cesm/development/cross-wg/S2S/jaye/"+verifdata+".tas.zarr/", consolidated=True)
land = xr.open_dataset("/glade/work/jaye/landsea.nc")

In [5]:
# converting to a proleptic gregorian calendar since the observations
# have leap years. you could also just drop the leap year days from
# the obs.
hinda2["init"] = [cftime.DatetimeProlepticGregorian(d.year, d.month, d.day) for d in hinda2["init"].values]
hinda1["init"] = [cftime.DatetimeProlepticGregorian(d.year, d.month, d.day) for d in hinda1["init"].values]
hindaw["init"] = [cftime.DatetimeProlepticGregorian(d.year, d.month, d.day) for d in hindaw["init"].values]

# designate the lead time step unit.
hinda2["lead"].attrs["units"] = "days"
hinda1["lead"].attrs["units"] = "days"
hindaw["lead"].attrs["units"] = "days"

In [6]:
# Convert our verification product to the same calendar type.
# Climpred just needs to see the same datetime object styles for
# alignment. You could use numpy datestrings or pandas on both,
# cftime.noleap or gregorian, etc.
def convert_to_cftime(x):
    x = str(x)
    y, m, d = x.split("-")
    y = int(y)
    m = int(m)
    d = int(d.split("T")[0])
    return cftime.DatetimeProlepticGregorian(y, m, d)


verif["time"] = [convert_to_cftime(t) for t in verif.time.values]

In [7]:
hindaw = hindaw.sel(init=slice("1999-01-01", "2015-12-31"))
hinda1 = hinda1.sel(init=slice("1999-01-01", "2015-12-31"))
hinda2 = hinda2.sel(init=slice("1999-01-01", "2015-12-31"))
verif = verif.sel(time=slice("1999-01-01","2015-12-31"))

In [8]:
#Weird stuff in new zealand happening. This is easiest way to fix that i have found.
if verifdata=="CPC":
    verif = verif.where(-100<verif.TAS)
    verif = verif.where(100>verif.TAS)

In [9]:
verif = verif.isel(lat=slice(0,-1))
hinda2 = hinda2.isel(lat=slice(0,-1))
hinda1 = hinda1.isel(lat=slice(0,-1))
hindaw = hindaw.isel(lat=slice(0,-1))
verif.coords['mask'] = (("lat","lon"),land.LSMASK)
hinda2.coords['mask'] = (("lat","lon"),land.LSMASK)
hinda1.coords['mask'] = (("lat","lon"),land.LSMASK)
hindaw.coords['mask'] = (("lat","lon"),land.LSMASK)

In [10]:
if domain == "North America":
    hinda2 = hinda2.sel(lat=slice(15.0, 75.0))
    hinda2 = hinda2.sel(lon=slice(190.0, 305.0))
    hinda1 = hinda1.sel(lat=slice(15.0, 75.0))
    hinda1 = hinda1.sel(lon=slice(190.0, 305.0))
    hindaw = hindaw.sel(lat=slice(15.0, 75.0))
    hindaw = hindaw.sel(lon=slice(190.0, 305.0))
    verif = verif.sel(lat=slice(15.0, 75.0))
    verif = verif.sel(lon=slice(190.0, 305.0))

In [11]:
hinda2 = hinda2.where(hinda2.mask==1)
hinda1 = hinda1.where(hinda1.mask==1)
hindaw = hindaw.where(hindaw.mask==1)
verif = verif.where(verif.mask==1)

In [12]:
verif = verif.chunk({"time": -1, "lat": "auto", "lon": "auto"}).persist()
hindaw = hindaw.chunk({"member": -1, "init": -1, "lead": "auto", "lat": 36, "lon": 72}).persist()
hinda1 = hinda1.chunk({"member": -1, "init": -1, "lead": "auto", "lat": 36, "lon": 72}).persist()
hinda2 = hinda2.chunk({"member": -1, "init": -1, "lead": "auto", "lat": 36, "lon": 72}).persist()

In [13]:
hindaw_djf = hindaw.sel(init=hindaw.init.dt.season=="DJF")
hinda1_djf = hinda1.sel(init=hinda1.init.dt.season=="DJF")
hinda1_jja = hinda1.sel(init=hinda1.init.dt.season=="JJA")
hinda2_djf = hinda2.sel(init=hinda2.init.dt.season=="DJF")
hinda2_jja = hinda2.sel(init=hinda2.init.dt.season=="JJA")
verif_djf = verif.sel(time=verif.time.dt.season=="DJF")
verif_jja = verif.sel(time=verif.time.dt.season=="JJA")

In [14]:
hinda1_5_djf = hinda1_djf.sel(member = slice(0,4))
hinda2_5_djf = hinda2_djf.sel(member = slice(0,4))
hinda1_5_jja = hinda1_jja.sel(member = slice(0,4))
hinda2_5_jja = hinda2_jja.sel(member = slice(0,4))

In [15]:
#Calculate ensemble spread
ensprw_djf = hindaw_djf.TAS.std("member").weighted(np.cos(np.deg2rad(hindaw_djf.lat))).mean(("lat","lon")).mean("init")
enspr1_djf = hinda1_djf.TAS.std("member").weighted(np.cos(np.deg2rad(hinda1_djf.lat))).mean(("lat","lon")).mean("init")
enspr2_djf = hinda2_djf.TAS.std("member").weighted(np.cos(np.deg2rad(hinda2_djf.lat))).mean(("lat","lon")).mean("init")
enspr1_5_djf = hinda1_5_djf.TAS.std("member").weighted(np.cos(np.deg2rad(hinda1_5_djf.lat))).mean(("lat","lon")).mean("init")
enspr2_5_djf = hinda2_5_djf.TAS.std("member").weighted(np.cos(np.deg2rad(hinda2_5_djf.lat))).mean(("lat","lon")).mean("init")
enspr1_jja = hinda1_jja.TAS.std("member").weighted(np.cos(np.deg2rad(hinda1_jja.lat))).mean(("lat","lon")).mean("init")
enspr2_jja = hinda2_jja.TAS.std("member").weighted(np.cos(np.deg2rad(hinda2_jja.lat))).mean(("lat","lon")).mean("init")
enspr1_5_jja = hinda1_5_jja.TAS.std("member").weighted(np.cos(np.deg2rad(hinda1_5_jja.lat))).mean(("lat","lon")).mean("init")
enspr2_5_jja = hinda2_5_jja.TAS.std("member").weighted(np.cos(np.deg2rad(hinda2_5_jja.lat))).mean(("lat","lon")).mean("init")

In [16]:
#Here starts Judith's code after Aaron's example

In [17]:
def my_rps(
    observations,
    forecasts,
    category_edges,
    dim=None,
    fair=False,
    weights=None,
    keep_attrs=False,
    member_dim="member",
    climatology_for_quantile=None,
):
    """Calculate Ranked Probability Score.
     .. math::
        RPS(p, k) = 1/M \\sum_{m=1}^{M}
        [(\\sum_{k=1}^{m} p_k) - (\\sum_{k=1}^{m} o_k)]^{2}
    Parameters
    ----------
    observations : xarray.Dataset or xarray.DataArray
        The observations or set of observations of the event.
        Data should be boolean or logical \
        (True or 1 for event occurance, False or 0 for non-occurance).
    forecasts : xarray.Dataset or xarray.DataArray
        The forecast likelihoods of the event.
        If ``fair==False``, forecasts should be between 0 and 1 without a dimension
        ``member_dim`` or should be boolean (True,False) or binary (0, 1) containing a
        member dimension (probabilities will be internally calculated by
        ``.mean(member_dim))``. If ``fair==True``, forecasts must be boolean
        (True,False) or binary (0, 1) containing dimension ``member_dim``.
    category_edges : array_like
        Category bin edges used to compute the CDFs. Similar to np.histogram, \
        all but the last (righthand-most) bin include the left edge and exclude \
        the right edge. The last bin includes both edges.
    dim : str or list of str, optional
        Dimension over which to compute mean after computing ``rps``.
        Defaults to None implying averaging over all dimensions.
    fair: boolean
        Apply ensemble member-size adjustment for unbiased, fair metric;
        see Ferro (2013). Defaults to False.
    weights : xr.DataArray with dimensions from dim, optional
        Weights for `weighted.mean(dim)`. Defaults to None, such that no weighting is
        applied.
    keep_attrs : bool
        If True, the attributes (attrs) will be copied from the first input to the new
        one. If False (default), the new object will be returned without attributes.
    member_dim : str, optional
        Name of ensemble member dimension. By default, 'member'.
    Returns
    -------
    xarray.Dataset or xarray.DataArray:
        ranked probability score
    Examples
    --------
    >>> observations = xr.DataArray(np.random.normal(size=(3,3)),
    ...                             coords=[('x', np.arange(3)),
    ...                                     ('y', np.arange(3))])
    >>> forecasts = xr.DataArray(np.random.normal(size=(3,3,3)),
    ...                          coords=[('x', np.arange(3)),
    ...                                  ('y', np.arange(3)),
    ...                                  ('member', np.arange(3))])
    >>> category_edges = np.array([.2, .5, .8])
    >>> rps(observations > 0.5, (forecasts > 0.5).mean('member'), category_edges)
    <xarray.DataArray 'histogram_category' (y: 3)>
    array([1.        , 1.        , 0.33333333])
    Coordinates:
      * y        (y) int64 0 1 2
    References
    ----------
    * https://www.cawcr.gov.au/projects/verification/verif_web_page.html#RPS
    * C. A. T. Ferro. Fair scores for ensemble forecasts. Q.R.J. Meteorol. Soc., 140:
      1917–1923, 2013. doi: 10.1002/qj.2270.
    * https://www-miklip.dkrz.de/about/problems/
    """
    bin_names = ["category"]
    bin_dim = f"{bin_names[0]}_bin"
    M = forecasts[member_dim].size
    if climatology_for_quantile is None:
        # histogram(dim=[]) not allowed therefore add fake member dim
        # to apply over when multi-dim observations
        if len(observations.dims) == 1:
            observations = histogram(
                observations, bins=[category_edges], bin_names=bin_names, dim=None
            )
        else:
            observations = histogram(
                observations.expand_dims(member_dim),
                bins=[category_edges],
                bin_names=bin_names,
                dim=[member_dim],
            )

        forecasts = histogram(
            forecasts,
            bins=[category_edges],
            bin_names=bin_names,
            dim=[member_dim],
        )
        if fair:
            e = forecasts

        # normalize f.sum()=1
        forecasts = forecasts / forecasts.sum(bin_dim)
        observations = observations / observations.sum(bin_dim)

        Fc = forecasts.cumsum(bin_dim)
        Oc = observations.cumsum(bin_dim)
        
    else:
        if isinstance(climatology_for_quantile, (xr.Dataset, xr.DataArray)):
            if type(climatology_for_quantile) == type(forecasts):
                boundaries = climatology_for_quantile.quantile(q=category_edges, dim='time').rename({'quantile':bin_dim})
                Fc = (forecasts < boundaries).mean(member_dim)
                Oc = (observations < boundaries)
                # todo: mask land
            else:
                raise ValueError('must be as forecast and obs')
        else:
            raise ValueError('must be xarray object')

    if fair:
        Ec = e.cumsum(bin_dim)
        res = (((Ec / M) - Oc) ** 2 - Ec * (M - Ec) / (M ** 2 * (M - 1))).sum(bin_dim)
    else:
        res = ((Fc - Oc) ** 2).sum(bin_dim)
    if weights is not None:
        res = res.weighted(weights)
    res = xr.apply_ufunc(np.clip, res, 0, 1, dask="allowed")  # dirty fix
    return res.mean(dim, keep_attrs=keep_attrs)


def my_rps(
    observations,
    forecasts,
    category_edges,
    dim=None,
    fair=False,
    weights=None,
    keep_attrs=False,
    member_dim="member",
    climatology_for_quantile=None,
):
    """Calculate Ranked Probability Score.
     .. math::
        RPS(p, k) = 1/M \\sum_{m=1}^{M}
        [(\\sum_{k=1}^{m} p_k) - (\\sum_{k=1}^{m} o_k)]^{2}
    Parameters
    ----------
    observations : xarray.Dataset or xarray.DataArray
        The observations or set of observations of the event.
        Data should be boolean or logical \
        (True or 1 for event occurance, False or 0 for non-occurance).
    forecasts : xarray.Dataset or xarray.DataArray
        The forecast likelihoods of the event.
        If ``fair==False``, forecasts should be between 0 and 1 without a dimension
        ``member_dim`` or should be boolean (True,False) or binary (0, 1) containing a
        member dimension (probabilities will be internally calculated by
        ``.mean(member_dim))``. If ``fair==True``, forecasts must be boolean
        (True,False) or binary (0, 1) containing dimension ``member_dim``.
    category_edges : array_like
        Category bin edges used to compute the CDFs. Similar to np.histogram, \
        all but the last (righthand-most) bin include the left edge and exclude \
        the right edge. The last bin includes both edges.
    dim : str or list of str, optional
        Dimension over which to compute mean after computing ``rps``.
        Defaults to None implying averaging over all dimensions.
    fair: boolean
        Apply ensemble member-size adjustment for unbiased, fair metric;
        see Ferro (2013). Defaults to False.
    weights : xr.DataArray with dimensions from dim, optional
        Weights for `weighted.mean(dim)`. Defaults to None, such that no weighting is
        applied.
    keep_attrs : bool
        If True, the attributes (attrs) will be copied from the first input to the new
        one. If False (default), the new object will be returned without attributes.
    member_dim : str, optional
        Name of ensemble member dimension. By default, 'member'.
    Returns
    -------
    xarray.Dataset or xarray.DataArray:
        ranked probability score
    Examples
    --------
    >>> observations = xr.DataArray(np.random.normal(size=(3,3)),
    ...                             coords=[('x', np.arange(3)),
    ...                                     ('y', np.arange(3))])
    >>> forecasts = xr.DataArray(np.random.normal(size=(3,3,3)),
    ...                          coords=[('x', np.arange(3)),
    ...                                  ('y', np.arange(3)),
    ...                                  ('member', np.arange(3))])
    >>> category_edges = np.array([.2, .5, .8])
    >>> rps(observations > 0.5, (forecasts > 0.5).mean('member'), category_edges)
    <xarray.DataArray 'histogram_category' (y: 3)>
    array([1.        , 1.        , 0.33333333])
    Coordinates:
      * y        (y) int64 0 1 2
    References
    ----------
    * https://www.cawcr.gov.au/projects/verification/verif_web_page.html#RPS
    * C. A. T. Ferro. Fair scores for ensemble forecasts. Q.R.J. Meteorol. Soc., 140:
      1917–1923, 2013. doi: 10.1002/qj.2270.
    * https://www-miklip.dkrz.de/about/problems/
    """
    bin_names = ["category"]
    bin_dim = f"{bin_names[0]}_bin"
    M = forecasts[member_dim].size
    
    if isinstance(category_edges, (xr.Dataset, xr.DataArray)):
        #print('new way')
        if type(category_edges) == type(forecasts):
            if 'quantile' in category_edges.dims:
                category_edges=category_edges.rename({'quantile':bin_dim})
            Fc = (forecasts < category_edges).mean(member_dim)
            Oc = (observations < category_edges)
            # todo: mask land
        else:
            raise ValueError('must be as forecast and obs')

    elif isinstance(category_edges, (np.ndarray,np.array)):
        #print('old way')
        # histogram(dim=[]) not allowed therefore add fake member dim
        # to apply over when multi-dim observations
        if len(observations.dims) == 1:
            observations = histogram(
                observations, bins=[category_edges], bin_names=bin_names, dim=None
            )
        else:
            observations = histogram(
                observations.expand_dims(member_dim),
                bins=[category_edges],
                bin_names=bin_names,
                dim=[member_dim],
            )

        forecasts = histogram(
            forecasts,
            bins=[category_edges],
            bin_names=bin_names,
            dim=[member_dim],
        )
        if fair:
            e = forecasts

        # normalize f.sum()=1
        forecasts = forecasts / forecasts.sum(bin_dim)
        observations = observations / observations.sum(bin_dim)

        Fc = forecasts.cumsum(bin_dim)
        Oc = observations.cumsum(bin_dim)
    else:
        raise ValueError(f'category_edges must be xr.DataArray, xr.Dataset or np.array, found {type(category_edges)}')

    if fair:
        Ec = e.cumsum(bin_dim)
        res = (((Ec / M) - Oc) ** 2 - Ec * (M - Ec) / (M ** 2 * (M - 1))).sum(bin_dim)
    else:
        res = ((Fc - Oc) ** 2).sum(bin_dim)
    if weights is not None:
        res = res.weighted(weights)
    res = xr.apply_ufunc(np.clip, res, 0, 1, dask="allowed")  # dirty fix
    return res.mean(dim, keep_attrs=keep_attrs)

In [18]:
def _rps(forecast, verif, dim=None, **metric_kwargs):
    """Ranked Probability Score.
    .. math::
        RPS(p, k) = 1/M \\sum_{m=1}^{M} [(\\sum_{k=1}^{m} p_k) - (\\sum_{k=1}^{m} \
            o_k)]^{2}
    Args:
        forecast (xr.object): Raw forecasts with ``member`` dimension.
        verif (xr.object): Verification data without ``member`` dim.
        dim (list or str): Dimensions to aggregate. Requires to contain `member`.
        category_edges (array_like): Category bin edges used to compute the CDFs.
            Bins include the left most edge, but not the right. Passed via
            metric_kwargs.
    Details:
        +-----------------+-----------+
        | **minimum**     | 0.0       |
        +-----------------+-----------+
        | **maximum**     | 1.0       |
        +-----------------+-----------+
        | **perfect**     | 0.0       |
        +-----------------+-----------+
        | **orientation** | negative  |
        +-----------------+-----------+
    See also:
        * :py:func:`~xskillscore.rps`
    Example:
        >>> category_edges = np.array([-.5, 0., .5, 1.])
        >>> hindcast.verify(metric='rps', comparison='m2o', dim='member',
                alignment='same_verifs', category_edges=category_edges)
        >>> perfect_model.verify(metric='rps', comparison='m2c',
                dim='member', category_edges=category_edges)
    """
    dim = _remove_member_from_dim_or_raise(dim)
    if "category_edges" in metric_kwargs:
        category_edges = metric_kwargs.pop("category_edges")
    else:
        raise ValueError("require category_edges")
    return my_rps(verif, forecast, category_edges, dim=dim, **metric_kwargs)

from climpred.metrics import Metric, _remove_member_from_dim_or_raise
__rps = Metric(
    name="rps",
    function=_rps,
    positive=False,
    probabilistic=True,
    unit_power=0,
    long_name="rps",
    minimum=0.0,
    maximum=1.0,
    perfect=0.0,
)

In [19]:
hindcastw_djf = climpred.HindcastEnsemble(hindaw_djf).add_observations(verif_djf)
hindcast1_djf = climpred.HindcastEnsemble(hinda1_djf).add_observations(verif_djf)
hindcast2_djf = climpred.HindcastEnsemble(hinda2_djf).add_observations(verif_djf)
hindcast1_5_djf = climpred.HindcastEnsemble(hinda1_5_djf).add_observations(verif_djf)
hindcast2_5_djf = climpred.HindcastEnsemble(hinda2_5_djf).add_observations(verif_djf)
hindcast1_jja = climpred.HindcastEnsemble(hinda1_jja).add_observations(verif_jja)
hindcast2_jja = climpred.HindcastEnsemble(hinda2_jja).add_observations(verif_jja)
hindcast1_5_jja = climpred.HindcastEnsemble(hinda1_5_jja).add_observations(verif_jja)
hindcast2_5_jja = climpred.HindcastEnsemble(hinda2_5_jja).add_observations(verif_jja)

In [20]:
#(hinda2_djf<terciles_boundaries_djf).mean('member').isel(init=1,lead=2).TAS.plot(col='quantile', yincrease=True)

In [21]:
#pm = climpred.PerfectModelEnsemble(init).add_control(control)
#rps_result = pm.verify(metric=__rps, comparison='m2c', dim=['init','member'], category_edges=control.quantile(q=[.33,.66], dim='time'))
# skill = hindcast.verify(
#        metric="rmse",
#        alignment="maximize",
#        comparison="e2o",
#        dim="init",
#        reference="persistence",
#    )

In [22]:
bins=verif_djf.quantile(q=[.33,.66], dim=['time'])
rps2_djf = hindcast2_djf.verify(metric=__rps, comparison=comp, dim=dim, alignment='maximize',category_edges=bins)
#rps2_djf = hindcast2_djf.verify(metric=__rps, comparison=comp, dim=dim, alignment='maximize',category_edges=bins, reference='persistence')

In [23]:
hindcast2_djf.verify


climpred.HindcastEnsemble

<Initialized Ensemble>
Dimensions: (init: 219, lat: 61, lead: 6, lon: 116, member: 11)
Coordinates:
 dayofyear (init) int64 dask.array<chunksize=(219,), meta=np.ndarray>
 * init (init) object 1999-01-04 00:00:00 ... 2015-12-28 00:00:00
 * lat (lat) float32 15.0 16.0 17.0 18.0 19.0 ... 72.0 73.0 74.0 75.0
 * lead (lead) int64 0 1 2 3 4 5
 * lon (lon) float32 190.0 191.0 192.0 193.0 ... 302.0 303.0 304.0 305.0
 * member (member) int64 1 2 3 4 5 6 7 8 9 10 11
 mask (lat, lon) int8 dask.array<chunksize=(36, 72), meta=np.ndarray>
Data variables:
 TAS (member, init, lead, lat, lon) float32 dask.array<chunksize=(11, 219, 1, 36, 72), meta=np.ndarray> Initialized Ensemble Dimensions: init : 219 lat : 61 lead : 6 lon : 116 member : 11 Coordinates: (7) dayofyear (init) int64 dask.array<chunksize=(219,), meta=np.ndarray> 
 
 
 
 
 Array Chunk 
 
 
 Bytes 1.75 kB 1.75 kB 
 Shape (219,) (219,) 
 Count 2 Tasks 1 Chunks 
 Type int64 numpy.ndarray 
 
 
 
 
 

 <!-- Horizontal lines -->
 
 

 <!-- Vertical lines -->
 
 

 <!-- Colored Rectangle -->
 

 <!-- Text -->
 219 
 1 
 
 
 
 init (init) object 1999-01-04 00:00:00 ... 2015-12-... array([cftime.DatetimeProlepticGregorian(1999, 1, 4, 0, 0, 0, 0),
 cftime.DatetimeProlepticGregorian(1999, 1, 11, 0, 0, 0, 0),
 cftime.DatetimeProlepticGregorian(1999, 1, 18, 0, 0, 0, 0), ...,
 cftime.DatetimeProlepticGregorian(2015, 12, 14, 0, 0, 0, 0),
 cftime.DatetimeProlepticGregorian(2015, 12, 21, 0, 0, 0, 0),
 cftime.DatetimeProlepticGregorian(2015, 12, 28, 0, 0, 0, 0)],
 dtype=object) lat (lat) float32 15.0 16.0 17.0 ... 73.0 74.0 75.0 array([15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42.,
 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55., 56.,
 57., 58., 59., 60., 61., 62., 63., 64., 65., 66., 67., 68., 69., 70.,
 71., 72., 73., 74., 75.], dtype=float32) lead (lead) int64 0 1 2 3 4 5 units : days array([0, 1, 2, 3, 4, 5]) lon (lon) float32 190.0 191.0 192.0 ... 304.0 305.0 array([190., 191., 192., 193., 194., 195., 196., 197., 198., 199., 200., 201.,
 202., 203., 204., 205., 206., 207., 208., 209., 210., 211., 212., 213.,
 214., 215., 216., 217., 218., 219., 220., 221., 222., 223., 224., 225.,
 226., 227., 228., 229., 230., 231., 232., 233., 234., 235., 236., 237.,
 238., 239., 240., 241., 242., 243., 244., 245., 246., 247., 248., 249.,
 250., 251., 252., 253., 254., 255., 256., 257., 258., 259., 260., 261.,
 262., 263., 264., 265., 266., 267., 268., 269., 270., 271., 272., 273.,
 274., 275., 276., 277., 278., 279., 280., 281., 282., 283., 284., 285.,
 286., 287., 288., 289., 290., 291., 292., 293., 294., 295., 296., 297.,
 298., 299., 300., 301., 302., 303., 304., 305.], dtype=float32) member (member) int64 1 2 3 4 5 6 7 8 9 10 11 array([ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]) mask (lat, lon) int8 dask.array<chunksize=(36, 72), meta=np.ndarray> 
 
 
 
 
 Array Chunk 
 
 
 Bytes 7.08 kB 2.59 kB 
 Shape (61, 116) (36, 72) 
 Count 4 Tasks 4 Chunks 
 Type int8 numpy.ndarray 
 
 
 
 
 

 <!-- Horizontal lines -->
 
 
 

 <!-- Vertical lines -->
 
 
 

 <!-- Colored Rectangle -->
 

 <!-- Text -->
 116 
 61 
 
 
 
 Data variables: (1) TAS (member, init, lead, lat, lon) float32 dask.array<chunksize=(11, 219, 1, 36, 72), meta=np.ndarray> 
 
 
 
 
 Array Chunk 
 
 
 Bytes 409.11 MB 24.98 MB 
 Shape (11, 219, 6, 61, 116) (11, 219, 1, 36, 72) 
 Count 48 Tasks 24 Chunks 
 Type float32 numpy.ndarray 
 
 
 
 
 

 <!-- Horizontal lines -->
 
 

 <!-- Vertical lines -->
 
 

 <!-- Colored Rectangle -->
 

 <!-- Text -->
 219 
 11 


 <!-- Horizontal lines -->
 
 
 

 <!-- Vertical lines -->
 
 
 
 
 
 
 

 <!-- Colored Rectangle -->
 

 <!-- Horizontal lines -->
 
 
 
 
 
 
 

 <!-- Vertical lines -->
 
 
 

 <!-- Colored Rectangle -->
 

 <!-- Horizontal lines -->
 
 
 

 <!-- Vertical lines -->
 
 
 

 <!-- Colored Rectangle -->
 

 <!-- Text -->
 116 
 61 
 6 
 
 
 
 Attributes: (0)

<Observations>
Dimensions: (lat: 61, lon: 116, time: 1534)
Coordinates:
 dayofyear (time) int64 dask.array<chunksize=(1534,), meta=np.ndarray>
 * lat (lat) float32 15.0 16.0 17.0 18.0 19.0 ... 72.0 73.0 74.0 75.0
 * lon (lon) float32 190.0 191.0 192.0 193.0 ... 302.0 303.0 304.0 305.0
 * time (time) object 1999-01-01 00:00:00 ... 2015-12-31 00:00:00
 mask (lat, lon) int8 dask.array<chunksize=(61, 116), meta=np.ndarray>
Data variables:
 TAS (time, lat, lon) float64 dask.array<chunksize=(1534, 45, 58), meta=np.ndarray> Observations Dimensions: lat : 61 lon : 116 time : 1534 Coordinates: (5) dayofyear (time) int64 dask.array<chunksize=(1534,), meta=np.ndarray> 
 
 
 
 
 Array Chunk 
 
 
 Bytes 12.27 kB 12.27 kB 
 Shape (1534,) (1534,) 
 Count 2 Tasks 1 Chunks 
 Type int64 numpy.ndarray 
 
 
 
 
 

 <!-- Horizontal lines -->
 
 

 <!-- Vertical lines -->
 
 

 <!-- Colored Rectangle -->
 

 <!-- Text -->
 1534 
 1 
 
 
 
 lat (lat) float32 15.0 16.0 17.0 ... 73.0 74.0 75.0 array([15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42.,
 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55., 56.,
 57., 58., 59., 60., 61., 62., 63., 64., 65., 66., 67., 68., 69., 70.,
 71., 72., 73., 74., 75.], dtype=float32) lon (lon) float32 190.0 191.0 192.0 ... 304.0 305.0 array([190., 191., 192., 193., 194., 195., 196., 197., 198., 199., 200., 201.,
 202., 203., 204., 205., 206., 207., 208., 209., 210., 211., 212., 213.,
 214., 215., 216., 217., 218., 219., 220., 221., 222., 223., 224., 225.,
 226., 227., 228., 229., 230., 231., 232., 233., 234., 235., 236., 237.,
 238., 239., 240., 241., 242., 243., 244., 245., 246., 247., 248., 249.,
 250., 251., 252., 253., 254., 255., 256., 257., 258., 259., 260., 261.,
 262., 263., 264., 265., 266., 267., 268., 269., 270., 271., 272., 273.,
 274., 275., 276., 277., 278., 279., 280., 281., 282., 283., 284., 285.,
 286., 287., 288., 289., 290., 291., 292., 293., 294., 295., 296., 297.,
 298., 299., 300., 301., 302., 303., 304., 305.], dtype=float32) time (time) object 1999-01-01 00:00:00 ... 2015-12-... array([cftime.DatetimeProlepticGregorian(1999, 1, 1, 0, 0, 0, 0),
 cftime.DatetimeProlepticGregorian(1999, 1, 2, 0, 0, 0, 0),
 cftime.DatetimeProlepticGregorian(1999, 1, 3, 0, 0, 0, 0), ...,
 cftime.DatetimeProlepticGregorian(2015, 12, 29, 0, 0, 0, 0),
 cftime.DatetimeProlepticGregorian(2015, 12, 30, 0, 0, 0, 0),
 cftime.DatetimeProlepticGregorian(2015, 12, 31, 0, 0, 0, 0)],
 dtype=object) mask (lat, lon) int8 dask.array<chunksize=(61, 116), meta=np.ndarray> 
 
 
 
 
 Array Chunk 
 
 
 Bytes 7.08 kB 7.08 kB 
 Shape (61, 116) (61, 116) 
 Count 1 Tasks 1 Chunks 
 Type int8 numpy.ndarray 
 
 
 
 
 

 <!-- Horizontal lines -->
 
 

 <!-- Vertical lines -->
 
 

 <!-- Colored Rectangle -->
 

 <!-- Text -->
 116 
 61 
 
 
 
 Data variables: (1) TAS (time, lat, lon) float64 dask.array<chunksize=(1534, 45, 58), meta=np.ndarray> 
 
 
 
 
 Array Chunk 
 
 
 Bytes 86.84 MB 32.03 MB 
 Shape (1534, 61, 116) (1534, 45, 58) 
 Count 8 Tasks 4 Chunks 
 Type float64 numpy.ndarray 
 
 
 
 
 

 <!-- Horizontal lines -->
 
 
 

 <!-- Vertical lines -->
 
 

 <!-- Colored Rectangle -->
 

 <!-- Horizontal lines -->
 
 

 <!-- Vertical lines -->
 
 
 

 <!-- Colored Rectangle -->
 

 <!-- Horizontal lines -->
 
 
 

 <!-- Vertical lines -->
 
 
 

 <!-- Colored Rectangle -->
 

 <!-- Text -->
 116 
 61 
 1534 
 
 
 
 Attributes: (0)

<bound method HindcastEnsemble.verify of >

In [25]:
bins = np.array([0.0,0.333,0.666])

In [ ]:
# Aaron - not clear why this is not working. Please help. 
# Also this is a hacked version using rps and np.array rather than percentiles, since __rps not 
# yet functional

In [27]:
skill = hindcast2_djf.verify(metric="rps",alignment="maximize",comparison=comp,dim=dim,reference="persistence",category_edges=bins)

ValueError: 'member' not found in array dimensions ('time', 'lat', 'lon')

In [ ]:
dim

In [ ]:
# Will imeplemnt here later (Abby's code)
if metric=="rps":

    category_edges=verif_djf.quantile(q=[.33,.66], dim=['time'])
    
    
    rpsw_djf = hindcastw_djf.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize', category_edges=bins)
    rps1_djf = hindcast1_djf.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize', category_edges=bins)
    rps2_djf = hindcast2_djf.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize', category_edges=bins)
    rps1_5_djf = hindcast1_5_djf.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize', category_edges=bins)
    rps2_5_djf = hindcast2_5_djf.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize', category_edges=bins)
    
    rps2_djf = hindcast2_djf.verify(metric=__rps, comparison=comp, dim=['init','member'], alignment='maximize',category_edges)
    
    rps1_jja = hindcast1_jja.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize', category_edges=bins)
    rps2_jja = hindcast2_jja.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize', category_edges=bins)
    rps1_5_jja = hindcast1_5_jja.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize', category_edges=bins)
    rps2_5_jja = hindcast2_5_jja.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize', category_edges=bins)
else:
    rpsw_djf = hindcastw_djf.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize')
    rps1_djf = hindcast1_djf.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize')
    rps2_djf = hindcast2_djf.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize')
    rps1_5_djf = hindcast1_5_djf.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize')
    rps2_5_djf = hindcast2_5_djf.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize')
    rps1_jja = hindcast1_jja.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize')
    rps2_jja = hindcast2_jja.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize')
    rps1_5_jja = hindcast1_5_jja.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize')
    rps2_5_jja = hindcast2_5_jja.verify(metric=metric, comparison=comp, dim=dim, alignment='maximize')

In [ ]:
rps2_djf = hindcast2_djf.verify(metric=__rps, comparison=comp, dim=['init','member'], alignment='maximize', category_edges=verif_djf.quantile(q=[.33,.66], dim=['time']))

In [ ]:
if metric=="rps":
    rpsw_djf = rpsw_djf.where(hindaw.mask==1)
    rps1_djf = rps1_djf.where(hindaw.mask==1)
    rps2_djf = rps2_djf.where(hindaw.mask==1)
    rps1_5_djf = rps1_5_djf.where(hindaw.mask==1)
    rps2_5_djf = rps2_5_djf.where(hindaw.mask==1)
    rps1_jja = rps1_jja.where(hindaw.mask==1)
    rps2_jja = rps2_jja.where(hindaw.mask==1)
    rps1_5_jja = rps1_5_jja.where(hindaw.mask==1)
    rps2_5_jja = rps2_5_jja.where(hindaw.mask==1)
    

In [ ]:
#rps2_djf.TAS.plot(col='lead')
#rps1_jja.TAS.sel(lead=slice(2,3)).mean("lead").plot()

In [ ]:
rpsw_djf_aw = rpsw_djf.weighted(np.cos(np.deg2rad(rpsw_djf.lat))).mean(("lat","lon")).persist()
rps1_djf_aw = rps1_djf.weighted(np.cos(np.deg2rad(rps1_djf.lat))).mean(("lat","lon")).persist()
rps2_djf_aw = rps2_djf.weighted(np.cos(np.deg2rad(rps2_djf.lat))).mean(("lat","lon")).persist()
rps1_5_djf_aw = rps1_5_djf.weighted(np.cos(np.deg2rad(rps1_5_djf.lat))).mean(("lat","lon")).persist()
rps2_5_djf_aw = rps2_5_djf.weighted(np.cos(np.deg2rad(rps2_5_djf.lat))).mean(("lat","lon")).persist()
rps1_jja_aw = rps1_jja.weighted(np.cos(np.deg2rad(rps1_djf.lat))).mean(("lat","lon")).persist()
rps2_jja_aw = rps2_jja.weighted(np.cos(np.deg2rad(rps2_djf.lat))).mean(("lat","lon")).persist()
rps1_5_jja_aw = rps1_5_jja.weighted(np.cos(np.deg2rad(rps1_5_djf.lat))).mean(("lat","lon")).persist()
rps2_5_jja_aw = rps2_5_jja.weighted(np.cos(np.deg2rad(rps2_5_djf.lat))).mean(("lat","lon")).persist()

In [ ]:
#Ensemble spread
#hinda2_djf.TAS.std("member").mean(("lat","lon")).mean("init").plot()

In [ ]:
#Ensemble mean if metric = 'RMSE"'
rps2_djf.TAS.mean(("lat", "lon")).plot()

In [ ]:
barWidth = 0.15
barsw = [rpsw_djf_aw.TAS.sel(lead=slice(0,1)).mean("lead").compute(),
         rpsw_djf_aw.TAS.sel(lead=slice(2,3)).mean("lead").compute(),
         rpsw_djf_aw.TAS.sel(lead=slice(4,5)).mean("lead").compute()]
rw = np.arange(len(barsw))
bars2_5 = [rps2_5_djf_aw.TAS.sel(lead=slice(0,1)).mean("lead").compute(),
           rps2_5_djf_aw.TAS.sel(lead=slice(2,3)).mean("lead").compute(),
           rps2_5_djf_aw.TAS.sel(lead=slice(4,5)).mean("lead").compute()]
r2_5 = [x + barWidth + 0.025 for x in rw]
bars2 = [rps2_djf_aw.TAS.sel(lead=slice(0,1)).mean("lead").compute(),
         rps2_djf_aw.TAS.sel(lead=slice(2,3)).mean("lead").compute(),
         rps2_djf_aw.TAS.sel(lead=slice(4,5)).mean("lead").compute()]
r2 = [x + barWidth + 0.025 for x in r2_5]
bars1_5 = [rps1_5_djf_aw.TAS.sel(lead=slice(0,1)).mean("lead").compute(),
           rps1_5_djf_aw.TAS.sel(lead=slice(2,3)).mean("lead").compute(),
           rps1_5_djf_aw.TAS.sel(lead=slice(4,5)).mean("lead").compute()]
r1_5 = [x + barWidth + 0.025 for x in r2]
bars1 = [rps1_djf_aw.TAS.sel(lead=slice(0,1)).mean("lead").compute(),
         rps1_djf_aw.TAS.sel(lead=slice(2,3)).mean("lead").compute(),
         rps1_djf_aw.TAS.sel(lead=slice(4,5)).mean("lead").compute()]
r1 = [x + barWidth + 0.025 for x in r1_5]

In [ ]:
plt.bar(rw, barsw, width = barWidth, color = "#2f70b7", edgecolor="white", label="CESM2-WACCM(5)")
plt.bar(r2_5, bars2_5, width = barWidth, color = "#d1a3ac", edgecolor="white", label="CESM2-CAM(5)")
plt.bar(r2, bars2, width = barWidth, color = "#942532", edgecolor="white", label="CESM2-CAM(10)")
plt.bar(r1_5, bars1_5, width = barWidth, color = "#c9e4a9", edgecolor="white", label="CESM1-CAM(5)")
plt.bar(r1, bars1, width = barWidth, color = "#7cbd3f", edgecolor="white", label="CESM1-CAM(10)")
plt.xticks([r + barWidth + 0.2 for r in range(len(bars1))], ["Weeks 1-2", "Weeks 3-4", "Weeks 5-6"])
plt.tick_params(axis = "y", direction = "in", which = "both")
plt.tick_params(axis = "y", which = "minor", left = False)
plt.tick_params(axis = "x", which = "both", bottom = False, top = False)
if metric=="acc":
    y_locator = FixedLocator([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]) #for rps and acc
    plt.gca().yaxis.set_major_locator(y_locator) #for rps and acc
    plt.ylim(0,1.0) #for rps and acc
elif metric=="rps":
    plt.ylim(0,1.5)
elif metric=="crpss":
    plt.ylim(-0.3,0.1) #for crpss
elif metric=="crps":
    plt.ylim(0,6.0) #for crps
elif metric=="rmse":
    plt.ylim(0,10.0) #for rmse
plt.ylabel(metric.upper())
plt.title("2m Temperature "+metric.upper()+" for "+domain+" land (DJF) - "+verifdata)
plt.legend(borderaxespad=0.6,edgecolor="black")
if domain == "North America":
    plt.savefig(metric+"_NAland_DJF_"+verifdata+".jpg",dpi=300,bbox_inches="tight")
elif domain == "global":
    plt.savefig(metric+"_globalland_DJF_"+verifdata+".jpg",dpi=300,bbox_inches="tight")

In [ ]:
barWidth = 0.175
bars2_5 = [rps2_5_jja_aw.TAS.sel(lead=slice(0,1)).mean("lead").persist(),
           rps2_5_jja_aw.TAS.sel(lead=slice(2,3)).mean("lead").persist(),
           rps2_5_jja_aw.TAS.sel(lead=slice(4,5)).mean("lead").persist()]
r2_5 = np.arange(len(bars2_5))
bars2 = [rps2_jja_aw.TAS.sel(lead=slice(0,1)).mean("lead").persist(),
         rps2_jja_aw.TAS.sel(lead=slice(2,3)).mean("lead").persist(),
         rps2_jja_aw.TAS.sel(lead=slice(4,5)).mean("lead").persist()]
r2 = [x + barWidth + 0.025 for x in r2_5]
bars1_5 = [rps1_5_jja_aw.TAS.sel(lead=slice(0,1)).mean("lead").persist(),
           rps1_5_jja_aw.TAS.sel(lead=slice(2,3)).mean("lead").persist(),
           rps1_5_jja_aw.TAS.sel(lead=slice(4,5)).mean("lead").persist()]
r1_5 = [x + barWidth + 0.025 for x in r2]
bars1 = [rps1_jja_aw.TAS.sel(lead=slice(0,1)).mean("lead").persist(),
         rps1_jja_aw.TAS.sel(lead=slice(2,3)).mean("lead").persist(),
         rps1_jja_aw.TAS.sel(lead=slice(4,5)).mean("lead").persist()]
r1 = [x + barWidth + 0.025 for x in r1_5]

In [ ]:
plt.bar(r2_5, bars2_5, width = barWidth, color = "#d1a3ac", edgecolor="white", label="CESM2-CAM(5)")
plt.bar(r2, bars2, width = barWidth, color = "#942532", edgecolor="white", label="CESM2-CAM(10)")
plt.bar(r1_5, bars1_5, width = barWidth, color = "#c9e4a9", edgecolor="white", label="CESM1-CAM(5)")
plt.bar(r1, bars1, width = barWidth, color = "#7cbd3f", edgecolor="white", label="CESM1-CAM(10)")
plt.xticks([r + barWidth + 0.2 for r in range(len(bars1))], ["Weeks 1-2", "Weeks 3-4", "Weeks 5-6"])
plt.tick_params(axis = "y", direction = "in", which = "both")
plt.tick_params(axis = "y", which = "minor", left = False)
plt.tick_params(axis = "x", which = "both", bottom = False, top = False)
if metric=="acc":
    y_locator = FixedLocator([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]) #for rps and acc
    plt.gca().yaxis.set_major_locator(y_locator) #for rps and acc
    plt.ylim(0,1.0) #for rps and acc
elif metric=="rps":
    plt.ylim(0,1.5)
elif metric=="crpss":
    plt.ylim(-0.3,0.1) #for crpss
elif metric=="crps":
    plt.ylim(0,6.0) #for crps
elif metric=="rmse":
    plt.ylim(0,10.0) #for rmse
plt.ylabel(metric.upper())
plt.title("2m Temperature "+metric.upper()+" for "+domain+" land (JJA) - "+verifdata)
plt.legend(borderaxespad=0.6,edgecolor="black")
if domain == "North America":
    plt.savefig(metric+"_NAland_JJA_"+verifdata+".jpg",dpi=300,bbox_inches="tight")
elif domain == "global":
    plt.savefig(metric+"_globalland_JJA_"+verifdata+".jpg",dpi=300,bbox_inches="tight")

Bar plots for with both ensemble spread and rmse

In [ ]:
ensprw_djf

In [ ]:
barWidth = 0.075
barsw = [rpsw_djf_aw.TAS.sel(lead=slice(0,1)).mean("lead").compute(),
         rpsw_djf_aw.TAS.sel(lead=slice(2,3)).mean("lead").compute(),
         rpsw_djf_aw.TAS.sel(lead=slice(4,5)).mean("lead").compute()]
rw = np.arange(len(barsw))
barsw_s = [ensprw_djf.sel(lead=slice(0,1)).mean("lead").compute(),
           ensprw_djf.sel(lead=slice(2,3)).mean("lead").compute(),
           ensprw_djf.sel(lead=slice(4,5)).mean("lead").compute()]
rw_s = [x + barWidth + 0.025 for x in rw]
bars2_5 = [rps2_5_djf_aw.TAS.sel(lead=slice(0,1)).mean("lead").compute(),
           rps2_5_djf_aw.TAS.sel(lead=slice(2,3)).mean("lead").compute(),
           rps2_5_djf_aw.TAS.sel(lead=slice(4,5)).mean("lead").compute()]
r2_5 = [x + barWidth + 0.025 for x in rw_s]
bars2_5_s = [enspr2_5_djf.sel(lead=slice(0,1)).mean("lead").compute(),
           enspr2_5_djf.sel(lead=slice(2,3)).mean("lead").compute(),
           enspr2_5_djf.sel(lead=slice(4,5)).mean("lead").compute()]
r2_5_s = [x + barWidth + 0.025 for x in r2_5]
bars2 = [rps2_djf_aw.TAS.sel(lead=slice(0,1)).mean("lead").compute(),
         rps2_djf_aw.TAS.sel(lead=slice(2,3)).mean("lead").compute(),
         rps2_djf_aw.TAS.sel(lead=slice(4,5)).mean("lead").compute()]
r2 = [x + barWidth + 0.025 for x in r2_5_s]
bars2_s = [enspr2_djf.sel(lead=slice(0,1)).mean("lead").compute(),
           enspr2_djf.sel(lead=slice(2,3)).mean("lead").compute(),
           enspr2_djf.sel(lead=slice(4,5)).mean("lead").compute()]
r2_s = [x + barWidth + 0.025 for x in r2]
bars1_5 = [rps1_5_djf_aw.TAS.sel(lead=slice(0,1)).mean("lead").compute(),
           rps1_5_djf_aw.TAS.sel(lead=slice(2,3)).mean("lead").compute(),
           rps1_5_djf_aw.TAS.sel(lead=slice(4,5)).mean("lead").compute()]
r1_5 = [x + barWidth + 0.025 for x in r2_s]
bars1_5_s = [enspr1_5_djf.sel(lead=slice(0,1)).mean("lead").compute(),
             enspr1_5_djf.sel(lead=slice(2,3)).mean("lead").compute(),
             enspr1_5_djf.sel(lead=slice(4,5)).mean("lead").compute()]
r1_5_s = [x + barWidth + 0.025 for x in r1_5]
bars1 = [rps1_djf_aw.TAS.sel(lead=slice(0,1)).mean("lead").compute(),
         rps1_djf_aw.TAS.sel(lead=slice(2,3)).mean("lead").compute(),
         rps1_djf_aw.TAS.sel(lead=slice(4,5)).mean("lead").compute()]
r1 = [x + barWidth + 0.025 for x in r1_5_s]
bars1_s = [enspr1_djf.sel(lead=slice(0,1)).mean("lead").compute(),
           enspr1_djf.sel(lead=slice(2,3)).mean("lead").compute(),
           enspr1_djf.sel(lead=slice(4,5)).mean("lead").compute()]
r1_s = [x + barWidth + 0.025 for x in r1]

In [ ]:
plt.bar(rw, barsw, width = barWidth, color = "#2f70b7", edgecolor="white", label="CESM2-WACCM(5)")
plt.bar(rw_s, barsw_s, width = barWidth, color = "#2f70b7", edgecolor="white", hatch="///")
plt.bar(r2_5, bars2_5, width = barWidth, color = "#d1a3ac", edgecolor="white", label="CESM2-CAM(5)")
plt.bar(r2_5_s, bars2_5_s, width = barWidth, color = "#d1a3ac", edgecolor="white", hatch="/")
plt.bar(r2, bars2, width = barWidth, color = "#942532", edgecolor="white", label="CESM2-CAM(10)")
plt.bar(r2_s, bars2_s, width = barWidth, color = "#942532", edgecolor="white", hatch="/")
plt.bar(r1_5, bars1_5, width = barWidth, color = "#c9e4a9", edgecolor="white", label="CESM1-CAM(5)")
plt.bar(r1_5_s, bars1_5_s, width = barWidth, color = "#c9e4a9", edgecolor="white", hatch="/")
plt.bar(r1, bars1, width = barWidth, color = "#7cbd3f", edgecolor="white", label="CESM1-CAM(10)")
plt.bar(r1_s, bars1_s, width = barWidth, color = "#7cbd3f", edgecolor="white", hatch="/")
plt.xticks([r + barWidth + 0.3 for r in range(len(bars1))], ["Weeks 1-2", "Weeks 3-4", "Weeks 5-6"])
plt.tick_params(axis = "y", direction = "in", which = "both")
plt.tick_params(axis = "y", which = "minor", left = False)
plt.tick_params(axis = "x", which = "both", bottom = False, top = False)
if metric=="acc":
    y_locator = FixedLocator([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]) #for rps and acc
    plt.gca().yaxis.set_major_locator(y_locator) #for rps and acc
    plt.ylim(0,1.0) #for rps and acc
elif metric=="rps":
    plt.ylim(0,1.5)
elif metric=="crpss":
    plt.ylim(-0.3,0.1) #for crpss
elif metric=="crps":
    plt.ylim(0,6.0) #for crps
elif metric=="rmse":
    plt.ylim(0,10.0) #for rmse
plt.ylabel(metric.upper())
plt.title("2m Temperature "+metric.upper()+" for "+domain+" land (DJF) - "+verifdata)
plt.legend(borderaxespad=0.6,edgecolor="black")
if domain == "North America":
    plt.savefig(metric+"_NAland_DJF_"+verifdata+".jpg",dpi=300,bbox_inches="tight")
elif domain == "global":
    plt.savefig(metric+"_globalland_DJF_"+verifdata+".jpg",dpi=300,bbox_inches="tight")